In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib

In [21]:
df = pd.read_csv("../data/real_final.csv")

In [22]:
# Target, feature 분리
y = df["Result"]
X = df.drop(columns=["Result"])

In [23]:
# 정규화 대상 컬럼과 인코딩 대상 분리
exclude_columns = ['GameDate', 'home_Team', 'away_Team']
X_to_scale = X.drop(columns=exclude_columns)

In [24]:
# 수치형 정규화
scaler = StandardScaler()
X_scaled_part = pd.DataFrame(scaler.fit_transform(X_to_scale), columns=X_to_scale.columns)

In [25]:
# 팀 정보 원-핫 인코딩
team_encoded = pd.get_dummies(
    X[["home_Team", "away_Team"]].reset_index(drop=True),
    columns=["home_Team", "away_Team"],
    prefix=["home_Team", "away_Team"]
).astype(int)

In [26]:
# 정규화된 수치 + 인코딩된 팀 정보 합치기
X_scaled = pd.concat([X_scaled_part.reset_index(drop=True), team_encoded], axis=1)

In [27]:
# 학습/검증 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=3
)

In [28]:
# Logistic Regression C 튜닝 반복문
C_values = np.logspace(-4, 2, 20)
train_scores = []
test_scores = []

for C in C_values:
    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train, y_train)
    train_scores.append(accuracy_score(y_train, model.predict(X_train)))
    test_scores.append(accuracy_score(y_test, model.predict(X_test)))

In [29]:
# 정확도 시각화
plt.figure(figsize=(10, 6))
plt.plot(C_values, train_scores, label='Train Accuracy', marker='o')
plt.plot(C_values, test_scores, label='Test Accuracy', marker='s')
plt.xscale('log')
plt.xlabel('C (Regularization Strength)')
plt.ylabel('Accuracy')
plt.title('Logistic Regression Performance by C')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [34]:
# 최적의 C 찾기
best_index = np.argmax(test_scores)
best_C = C_values[best_index]
print(f"최적의 C 값: {best_C}")

최적의 C 값: 0.007847599703514606


In [36]:
# 최적의 C로 모델 학습
model = LogisticRegression(C=best_C, max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(C=0.007847599703514606, max_iter=1000)

In [37]:
# 예측 및 평가
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix

y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

print("🎯 Accuracy:", accuracy_score(y_test, y_pred))
print("🎯 AUC:", roc_auc_score(y_test, y_proba))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# 🔽 여기에 Train/Test 비교 추가!
y_train_pred = model.predict(X_train)
y_train_proba = model.predict_proba(X_train)[:, 1]
y_test_pred = model.predict(X_test)
y_test_proba = model.predict_proba(X_test)[:, 1]

print("📘 Train Accuracy:", accuracy_score(y_train, y_train_pred))
print("📙 Test Accuracy :", accuracy_score(y_test, y_test_pred))
print("📘 Train AUC:", roc_auc_score(y_train, y_train_proba))
print("📙 Test AUC :", roc_auc_score(y_test, y_test_proba))

🎯 Accuracy: 0.898360655737705
🎯 AUC: 0.9625204512184621
              precision    recall  f1-score   support

           0       0.91      0.88      0.89       294
           1       0.89      0.92      0.90       316

    accuracy                           0.90       610
   macro avg       0.90      0.90      0.90       610
weighted avg       0.90      0.90      0.90       610

[[258  36]
 [ 26 290]]
📘 Train Accuracy: 0.8839688396883969
📙 Test Accuracy : 0.898360655737705
📘 Train AUC: 0.9541590358200915
📙 Test AUC : 0.9625204512184621


In [38]:
# 모델 + 스케일러 저장
joblib.dump(model, "./logistic_model.pkl")
print("✅ 모델 저장 완료!")

✅ 모델 저장 완료!
